<a href="https://colab.research.google.com/github/AhmedMM1122/Insect-task/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import Libraries
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import zscore
import pandas as pd
import time
import scipy.special
from scipy.special import gamma, gammaln
from scipy.stats import mode
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.decomposition import PCA
import scipy.io
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from scipy.linalg import block_diag
import argparse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load dataset
dataset = '/content/drive/MyDrive/INSECTS' #change this donia

fname1 = f'/content/drive/MyDrive/INSECTS/data.mat' #change this donia
fname2 = f'/content/drive/MyDrive/INSECTS/splits.mat'#change this donia

data = scipy.io.loadmat(fname1)
splits = scipy.io.loadmat(fname2)

In [ ]:
embeddings_dna = data['embeddings_dna']
embeddings_img = data['embeddings_img']
labels = data['labels']
trainval_loc = splits['trainval_loc']
train_loc = splits['train_loc']
test_seen_loc = splits['test_seen_loc']
test_unseen_loc = splits['test_unseen_loc']
val_seen_loc = splits['val_seen_loc']
val_unseen_loc = splits['val_unseen_loc']
G=data['G']

In [ ]:
model = 'OSBC_DNA'
transductive = True

In [ ]:
#Ridge_regression

def ridge_regression(embeddings_dna, embeddings_img, data_subset, lambda_,va_l1,va_l2):
    di = embeddings_img.shape[1]
    st = data_subset
    X = embeddings_img[st[va_l1], :].T
    D = embeddings_dna[st[va_l2], :].T

    DXT = np.dot(D, X.T)
    V = np.dot(DXT, np.linalg.inv(np.dot(X, X.T) + rho * np.eye(di)))

    return V

In [ ]:
if transductive:
  dd = embeddings_dna.shape[1]
  di = embeddings_img.shape[1]
  rho = 1
  if model == 'OSBC_IMG':
    print("Transductive model works well if the mapping is from Image to DNA,")
    print("Thus we automatically run this version!")
  model = 'OSBC_DIT'
  st = np.concatenate((trainval_loc, test_unseen_loc, test_seen_loc), axis=1)
  embeddings_dna = zscore(embeddings_dna, axis=0)
  embeddings_img = zscore(embeddings_img, axis=0)
  st = np.array(st).flatten()
  valid_indices = (st >= 0) & (st < embeddings_img.shape[0])
  valid_indices1 = (st >= 0) & (st < embeddings_dna.shape[0])
  start_time = time.time()
  V = ridge_regression(embeddings_dna, embeddings_img, st, rho,valid_indices,valid_indices1)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print('Time took for learning map in transductive setup:', elapsed_time)

Time took for learning map in transductive setup: 9.008608102798462


In [ ]:
pca_dim = 500

In [ ]:
def load_tuned_params(model):
 #We use the upper() method to convert the input model string to uppercase
    model = model.upper()
 #We define the parameter sets OSBC_IMG, OSBC_DNA, OSBC_DIL, and OSBC_DIT as lists.
    OSBC_IMG = [0.1, 10, 5 * pca_dim, 1]
    OSBC_DNA = [0.1, 10, 25 * pca_dim, 0.5]
    OSBC_DIL = [0.1, 10, 5 * pca_dim, 0.5]
    OSBC_DIT = [0.1, 10, 25 * pca_dim, 0.5]
#evluate model of data, if it is img,dna,.......
    if model == 'OSBC_IMG':
        data = OSBC_IMG
    elif model == 'OSBC_DNA':
        data = OSBC_DNA
    elif model == 'OSBC_DIL':
        data = OSBC_DIL
    elif model == 'OSBC_DIT':
        data = OSBC_DIT
    else:
        raise ValueError("Invalid model name")
#The selected parameters are unpacked into k_0, k_1, m, and s, and these values are returned as a tuple
    k_0, k_1, m, s = data
    return k_0, k_1, m, s


In [ ]:
k_0, k_1, m, s = load_tuned_params(model)

In [ ]:
tuning = False

In [ ]:
def data_split(embeddings_dna, embeddings_img, labels, trainval_loc, train_loc, test_seen_loc, test_unseen_loc, val_seen_loc, val_unseen_loc, split_type, model):
  if model == 'OSBC_IMG':
    features = embeddings_img
  else:
    features = embeddings_dna

  if split_type == 'tuning':
    train_idx = train_loc
    test_seen_idx = val_seen_loc
    test_unseen_idx = val_unseen_loc
  else:
    train_idx = trainval_loc
    test_seen_idx = test_seen_loc
    test_unseen_idx = test_unseen_loc

# Training data and labels
  xtrain = features[train_idx, :]
  ytrain = labels[train_idx]

# Test data and labels, Seen and Unseen
  valid_test_seen_idx = np.where(test_seen_idx < features.shape[0])
  valid_test_seen_idx = valid_test_seen_idx[0]  # Access the first element of the tuple
  xtest_seen = features[valid_test_seen_idx, :]
  ytest_seen = labels[valid_test_seen_idx]

  xtest_unseen = features[test_unseen_idx, :]
  ytest_unseen = labels[test_unseen_idx]

  return xtrain, ytrain, xtest_unseen, ytest_unseen, xtest_seen, ytest_seen


In [ ]:
x_tr, y_tr, x_ts_us, y_ts_us, x_ts_s, y_ts_s = data_split(embeddings_dna, embeddings_img, labels, trainval_loc, train_loc, test_seen_loc,
                                                          test_unseen_loc, val_seen_loc, val_unseen_loc, 'test', model);

In [ ]:
x_tr = x_tr.reshape(-1, 500)  # Reshape to (19420, 500)
y_tr = y_tr.reshape(-1, 19420)  # Reshape to (19420, 1)


In [ ]:
y_tr = y_tr.reshape(-1, 1)  # Reshape to (19420, 1)

In [ ]:
x_ts_us = x_ts_us.reshape(-1, 500)  # Reshape to (8463, 500)
y_ts_us = y_ts_us.reshape(-1, 8463)  # Reshape to (8463, 1)

In [ ]:
y_ts_us = y_ts_us.reshape(-1, 1)  # Reshape to (8463, 1)

In [ ]:
#function of  calculates the prior mean and prior covarince in advance to feed in Bayesian classifier
import numpy as np
import pandas as pd
#takes Data (the data matrix) and labels (class labels) as input arguments.
def calculate_priors(Data, labels):
    _, dim = Data.shape
    unique_labels = np.unique(labels)
    num_classes = len(unique_labels)

    scatters = np.zeros((dim, dim, num_classes))
    class_means = np.zeros((num_classes, dim))

    for j in range(num_classes):
        class_data = Data[labels == unique_labels[j]]
#We compute the covariance matrices for each class using np.cov.
#The rowvar=False argument specifies that each column represents a variable
        scatters[:, :, j] = np.cov(class_data, rowvar=False)
        class_means[j, :] = np.mean(class_data, axis=0)
#we calculate the overall scatter matrix as the mean of the per-class scatter matrices and the overall class mean as the mean of the class means.
    scatter = np.mean(scatters, axis=2)
    mu_0 = np.mean(class_means, axis=0)

    return mu_0, scatter



In [ ]:
Data = np.random.rand(100, 5)  # Example data with 100 samples and 5 features
labels = np.random.randint(0, 3, 100)  # Example labels (0, 1, or 2)

# Call the calculate_priors function with your data and labels
mu_0, scatter = calculate_priors(Data, labels)

# Print the calculated priors and scatter matrix
print("mu_0 (class means):")
print(mu_0)
print("\nScatter matrix:")
print(scatter)

mu_0 (class means):
[0.47394215 0.5046783  0.50473543 0.48172866 0.50269231]

Scatter matrix:
[[ 0.08532919 -0.01285999  0.00104571 -0.00758387 -0.01562938]
 [-0.01285999  0.08467357  0.00144442  0.00102159 -0.00532703]
 [ 0.00104571  0.00144442  0.10136072 -0.01351641 -0.02305694]
 [-0.00758387  0.00102159 -0.01351641  0.07885946  0.00225154]
 [-0.01562938 -0.00532703 -0.02305694  0.00225154  0.08744304]]


In [ ]:
import time
if transductive:
  #pca= 0 ,We use np.dot for matrix multiplication.
    x_tr_g = np.dot(V, embeddings_img[trainval_loc[0], :].T)
    x_tr_g = x_tr_g.T
    #we concatenate arrays using np.concatenate.
    x_tr = np.concatenate((x_tr, x_tr_g), axis=0)
    y_tr = np.concatenate((y_tr, y_tr))

# PCA for dimensionality reduction
pca_dim = 0
start_time = time.time()
C = np.cov(x_tr, rowvar=False)
#computes its eigenvalues and eigenvectors (vv)
_, vv = np.linalg.eig(C)
x_tr = np.dot(x_tr, vv[:, -pca_dim:])
x_ts_s = np.dot(x_ts_s, vv[:, -pca_dim:])
x_ts_us = np.dot(x_ts_us, vv[:, -pca_dim:])
end_time = time.time()  # Record the end time

# Calculate and print the time taken for PCA
pca_time = end_time - start_time
print(f"Time for PCA: {pca_time} seconds")
print (embeddings_img[trainval_loc, :].shape)

Time for PCA: 1.7472145557403564 seconds
(1, 19420, 2048)


In [ ]:
import argparse

def hyperparameter_setting(params):
    parser = argparse.ArgumentParser(description="Parse hyperparameters")
    dim = 500

    # Default values for hyperparameters
    default_k0 = 0.01
    default_k1 = 10
    default_m = 5 * dim
    default_mu0 = 0
    default_s = 1
    default_iter = 1
    default_pca = 0
    default_scatter = 0
    default_tuning = False

    parser.add_argument('--kappa_0', type=float, default=default_k0)
    parser.add_argument('--kappa_1', type=float, default=default_k1)
    parser.add_argument('--cov_shape', type=float, default=default_m)
    parser.add_argument('--prior_mean', type=float, default=default_mu0)
    parser.add_argument('--prior_covscale', type=float, default=default_s)
    parser.add_argument('--iter', type=int, default=default_iter)
    parser.add_argument('--pca', type=int, default=default_pca)
    parser.add_argument('--scatter', type=int, default=default_scatter)
    parser.add_argument('--tuning', type=bool, default=default_tuning)

    args = parser.parse_args(params)

    k_0 = args.kappa_0
    k_1 = args.kappa_1
    m = args.cov_shape
    num_iter = args.iter
    pca_dim = args.pca
    mu_0 = args.prior_mean
    s = args.prior_covscale
    scatter = args.scatter
    tuning = args.tuning

    if mu_0 == 0:
        mu_0 = [0] * dim
    elif scatter == 0:
        scatter = [[1 if i == j else 0 for j in range(dim)] for i in range(dim)]

    return k_0, k_1, m, mu_0, s, scatter, num_iter, pca_dim, tuning

In [ ]:
parameters = {
    'kappa_0': k_0,
    'kappa_1': k_1,
    'cov_shape': m,
    'prior_covscale': s,
    'pca': pca_dim
    # Add other parameters as needed
}

In [ ]:
arg_list = [f'--{key} {value}' for key, value in parameters.items()]

In [ ]:
arg_list = []
for key, value in parameters.items():
    arg_list.extend(['--' + key, str(value)])


In [ ]:
k_0, k_1, m, mu_0, s, scatter, num_iter, pca_dim, tuning = hyperparameter_setting(arg_list)

In [ ]:
import numpy as np
#The ppd_derivation function appears to calculate class predictive statistics (such as covariances, means, and degrees of freedom)
# for a dataset with class labels and genus labels
def ppd_derivation(X, Y, G, Psi, mu0, m, k0, k1): #X -> The dataset, where each row represents a data point, and each column represents a feature.
#Y -> Class labels for each data point in X.
#G ->enus labels for each data point in X.
#Psi -> Initial covariance matrix.
#mu0 -> Initial mean vector.
#m, k0, k1 -> Hyperparameters used in the calculations.
    seenclasses = np.unique(Y)
#Separates seen classes (those with class labels) from surrogate classes (those without class labels) based on the provided genus labels.
    seengenera = G[seenclasses]
    uyg = np.unique(G)
    ct = len(uyg)
    nc = len(seenclasses) + ct
    n, d = X.shape
#Sig_s -> Class predictive covariances for each class (seen and surrogate).
#mu_s -> Class predictive means for each class.
#v_s -> Class predictive degrees of freedom for each class.
#class_id -> Class IDs indicating which class each set of statistics belongs to.
#Sigmas -> Intermediate results for covariance calculations.
    Sig_s = np.zeros((d, d, nc))
    Sigmas = np.zeros((d, d, nc))
    mu_s = np.zeros((nc, d))
    v_s = np.zeros(nc)

    uy = seenclasses
    ncl = len(uy)
    cnt = 0
#For each seen class:

#1-Computes class-specific statistics, including the number of points, mean, and scatter matrix.
#2-Calculates class predictive statistics, taking into account data likelihood and global prior.
    for i in range(ncl):
        in_idx = Y == uy[i]
        Xi = X[in_idx, :]

        cur_n = np.sum(in_idx)
        cur_S = (cur_n - 1) * np.cov(Xi, rowvar=False)
        cur_mu = np.mean(Xi, axis=0)

        v_s[cnt] = cur_n + m - d + 1
        mu_s[cnt, :] = (cur_n * cur_mu + (k0 * k1 / (k0 + k1)) * mu0) / (cur_n + (k0 * k1 / (k0 + k1)))
        Smu = ((cur_n * (k0 * k1 / (k0 + k1))) / ((k0 * k1 / (k0 + k1)) + cur_n)) * np.outer(cur_mu - mu0, cur_mu - mu0)
        Sig_s[:, :, cnt] = (Psi + cur_S + Smu) / (((cur_n + (k0 * k1 / (k0 + k1))) * v_s[cnt]) / (cur_n + (k0 * k1 / (k0 + k1)) + 1))
        cnt += 1

    uy = np.unique(G)
    ncl = len(uy)
    class_id = np.zeros(nc)
    cnt = 0

    for i in range(ncl):
        in_idx = np.zeros(n, dtype=bool)

        genus = uy[i]
        classes = seenclasses[seengenera == genus]
        nci = len(classes)

        if nci >= 1:
#For each surrogate genus class:

#Identifies the classes belonging to the genus.
#Calculates component-specific statistics (means, scatter matrices, etc.) for the classes within the genus.
#Computes genus-specific predictive statistics based on the component statistics.
            for j in range(nci):
                in_idx[Y == classes[j]] = True

            Yi = Y[in_idx]
            Xi = X[in_idx, :]
            uyi = np.unique(Yi)

            ncpi = len(uyi)
            xkl = np.zeros((ncpi, d))
            Skl = np.zeros((d, d, ncpi))
            kap = np.zeros(ncpi)
            nkl = np.zeros(ncpi)

            for j in range(ncpi):
                in_idx = Yi == uyi[j]
                nkl[j] = np.sum(in_idx)
                kap[j] = nkl[j] * k1 / (nkl[j] + k1)
                Xij = Xi[in_idx, :]
                xkl[j, :] = np.mean(Xij, axis=0)
                Skl[:, :, j] = (nkl[j] - 1) * np.cov(Xij, rowvar=False)
#Collects the computed class predictive statistics, class IDs, and intermediate results for all classes (both seen and surrogate)
            sumkap = np.sum(kap)
            kaps = (sumkap + k0) * k1 / (sumkap + k0 + k1)
            sumSkl = np.sum(Skl, axis=2)
            muk = (np.sum(xkl * (kap.reshape(-1, 1)), axis=0) + k0 * mu0) / (np.sum(kap) + k0)
            vsc = np.sum(nkl) - ncpi + m - d + 1
            class_id[cnt] = uy[i]
            v_s[cnt] = vsc
            Sigmas[:, :, cnt] = Psi + sumSkl
            Sig_s[:, :, cnt] = (Psi + sumSkl) / ((kaps * v_s[cnt]) / (kaps + 1))
            mu_s[cnt, :] = muk
            cnt += 1

    return Sig_s, mu_s, v_s, class_id, Sigmas


In [ ]:
import numpy as np

# Generate example data
X = np.random.rand(100, 5)  # Sample data with 100 samples and 5 features
Y = np.random.randint(0, 3, 100)  # Class labels for 100 data points (3 classes)
G = np.random.randint(0, 2, 100)  # Genus labels for 100 data points (2 genera)
Psi = np.eye(5)  # Initial covariance matrix
mu0 = np.zeros(5)  # Initial mean vector
m = 2.0  # Hyperparameter m
k0 = 0.5  # Hyperparameter k0
k1 = 0.1  # Hyperparameter k1

# Call the ppd_derivation function
Sig_s, mu_s, v_s, class_id, Sigmas = ppd_derivation(X, Y, G, Psi, mu0, m, k0, k1)


In [ ]:
import numpy as np
import scipy.linalg  # Import scipy.linalg for Cholesky decomposition

def predict(X, Sig_s, mu_s, v_s, class_id):
    ncl, d = mu_s.shape
    piconst = (d / 2) * np.log(np.pi)
    gl_pc = scipy.special.gammaln(0.5 + np.arange(0.5, np.max(v_s) + d, 0.5))
    n = X.shape[0]
    prob_mat = np.zeros((n, ncl))

    epsilon = 1e-4  # Small positive value for regularization

    # Calculating log student-t likelihood for numerical stability
    for j in range(ncl):
        v = X - mu_s[j, :]  # Center the data

        Sig_s_j = Sig_s[:, :, j] + epsilon * np.eye(d)  # Apply regularization

        chsig = scipy.linalg.cholesky(Sig_s_j, lower=True)  # Cholesky decomposition
        tpar = gl_pc[int(v_s[j] + d)] - (gl_pc[int(v_s[j])] + (d / 2) * np.log(int(v_s[j])) + piconst) - np.sum(np.log(np.diag(chsig)))  # Stu-t likelihood part 1
        temp = scipy.linalg.solve_triangular(chsig, v.T, lower=True).T
        prob_mat[:, j] = tpar - 0.5 * (v_s[j] + d) * np.log(1 + (1 / v_s[j]) * np.sum(temp * temp, axis=1))

    bb = np.argmax(prob_mat, axis=1)
    ypred = class_id[bb]  # To ensure labels are correctly assigned back to original ones

    return ypred, prob_mat




In [ ]:
import numpy as np
import scipy.linalg

# Sample data (replace with your actual data)
X = np.random.rand(100, 2)  # Sample test data with 100 samples and 2 features
ncl = 3  # Number of classes
d = 2  # Number of features

# Sample class-specific parameters (replace with your actual parameters)
Sig_s = np.random.rand(d, d, ncl)  # Class predictive covariance matrices
mu_s = np.random.rand(ncl, d)  # Class predictive means
v_s = np.random.rand(ncl)  # Class predictive degrees of freedom
class_id = np.arange(1, ncl + 1)  # Class IDs

# Call the predict function with the sample data and parameters
ypred, prob_mat = predict(X, Sig_s, mu_s, v_s, class_id)

# Display the predicted labels and probability matrix
print("Predicted Labels:", ypred)
print("Probability Matrix:", prob_mat)


Predicted Labels: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Probability Matrix: [[inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf inf inf]
 [inf in

<ipython-input-102-5eaa66e1c67d>:20: RuntimeWarning: divide by zero encountered in log
  tpar = gl_pc[int(v_s[j] + d)] - (gl_pc[int(v_s[j])] + (d / 2) * np.log(int(v_s[j])) + piconst) - np.sum(np.log(np.diag(chsig)))  # Stu-t likelihood part 1


In [78]:

def evaluate(y_true, y_pred, G, mode):
    # Implement your evaluation logic here
    # Compute accuracy for each class
    unique_classes = np.unique(G)
    acc_per_class = []

    for c in unique_classes:
        class_indices = (G == c)
        class_accuracy = np.mean(y_true[class_indices] == y_pred[class_indices])
        acc_per_class.append(class_accuracy)

    overall_accuracy = np.mean(y_true == y_pred)

    return acc_per_class, overall_accuracy


In [80]:
import numpy as np

# Define your input data and labels (replace with your actual data)
y_true = np.array([0, 1, 1, 2, 2, 2, 3, 3, 4, 4])  # True class labels
y_pred = np.array([0, 1, 1, 2, 3, 2, 3, 3, 4, 4])  # Predicted class labels
G = np.array([0, 1, 1, 2, 2, 3, 3, 4, 4, 5])  # Genus labels
mode = 'seen'  # Replace with your specific mode

# Call the evaluate function
acc_per_class, overall_accuracy = evaluate(y_true, y_pred, G, mode)

# Print the results
print("Accuracy per Class:")
for i, acc in enumerate(acc_per_class):
    print(f"Class {i}: {acc * 100:.2f}%")

print(f"Overall Accuracy: {overall_accuracy * 100:.2f}%")


Accuracy per Class:
Class 0: 100.00%
Class 1: 100.00%
Class 2: 50.00%
Class 3: 100.00%
Class 4: 100.00%
Class 5: 100.00%
Overall Accuracy: 90.00%


In [87]:
import numpy as np

# Define helper functions here

def Bayesian_cls(x_tr, y_tr, x_ts_us, y_ts_us, x_ts_s, y_ts_s, G, *args):
    # Seen and unseen classes
    import time
    tic = time.time()
    d0 = x_tr.shape[1]

    # Parsing passed parameters and hyperparameters from tuning
    params = args
    k_0, k_1, m, mu_0, s, scatter, num_iter, pca_dim, tuning = hyperparameter_setting(params)

    # num_iter for repeating the procedure several times to eliminate
    # randomness
    # You may change the number of features to use by changing d0
    if pca_dim:
        # Dimensionality reduction from PCA
        C = np.cov(x_tr, rowvar=False)
        _, vv = np.linalg.eig(C)
        x_tr = x_tr.dot(vv[:, -pca_dim:])
        x_ts_s = x_ts_s.dot(vv[:, -pca_dim:])
        x_ts_us = x_ts_us.dot(vv[:, -pca_dim:])
        d0 = pca_dim

    # Main for loop for the calculations
    for iter in range(num_iter):
        # Training data
        xn = x_tr[:, fin[iter]]
        yn = y_tr

        # Test data from seen and unseen classes (GZSL)
        xt_unseen = x_ts_us[:, fin[iter]]
        xt_seen = x_ts_s[:, fin[iter]]

        # Pre-calculation of Psi (prior covariance) from tuned scale s, and
        # scatter. The reason behind this if statement is that we don't want
        # to repeat this precalculation in hypertuning since it is expensive
        # in time but we want to calculate this values with new data during
        # testing
        if tuning:
            Psi = (m - d0 - 1) * scatter / s
        else:
            mu_0, scatter = calculate_priors(xn, yn)
            Psi = (m - d0 - 1) * scatter / s

        # Class predictive cov, mean and DoF
        Sig_s, mu_s, v_s, class_id, Sigmas = ppd_derivation(xn, yn, G, Psi, mu_0, m, k_0, k_1)
        print('Model training time:')
        toc = time.time()
        print(toc - tic)

        # Inference phase
        tic = time.time()
        ypred_unseen = np.zeros((len(y_ts_us), num_iter), dtype=int)
        ypred_seen = np.zeros((len(y_ts_s), num_iter), dtype=int)
        print('Model inference time:')
        toc = time.time()
        print(toc - tic)

    # Performance calculation
    # Mode of iterations to alleviate the effect of random variables
    ypred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=ypred_unseen)
    acc_per_usclass, unseen_acc = evaluate(y_ts_us, ypred, G, 'unseen')  # Unseen classes

    # Accuracy calculation for seen classes
    ypred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=ypred_seen)
    acc_per_sclass, seen_acc = evaluate(y_ts_s, ypred, G, 'seen')  # Seen classes

    # Harmonic mean for seen and unseen classes acc. from Y. Xian paper
    H = 2 * unseen_acc * seen_acc / (unseen_acc + seen_acc)

    return seen_acc, unseen_acc, H, acc_per_sclass, acc_per_usclass

# Define helper functions hyperparameter_setting, calculate_priors, ppd_derivation, predict, and evaluate here

# Example usage:
# seen_acc, unseen_acc, H, acc_per_sclass, acc_per_usclass = Bayesian_cls(x_tr, y_tr, x_ts_us, y_ts_us, x_ts_s, y_ts_s, G, *params)
# print("Seen Accuracy:", seen_acc)
# print("Unseen Accuracy:", unseen_acc)
# print("Harmonic Mean (H):", H)
# print("Accuracy per Seen Class:", acc_per_sclass)
# print("Accuracy per Unseen Class:", acc_per_usclass)



In [90]:
import numpy as np

# Generate fabricated data for demonstration
np.random.seed(42)

# Training data and labels
x_tr = np.random.rand(100, 10)  # 100 samples, 10 features
y_tr = np.random.randint(0, 10, 100)  # 100 labels, assuming 10 classes

# Test data for unseen classes and labels
x_ts_us = np.random.rand(50, 10)  # 50 samples for unseen classes
y_ts_us = np.random.randint(10, 20, 50)  # 50 labels for unseen classes, assuming 10 unseen classes

# Test data for seen classes and labels
x_ts_s = np.random.rand(50, 10)  # 50 samples for seen classes
y_ts_s = np.random.randint(0, 10, 50)  # 50 labels for seen classes, assuming 10 seen classes

# Genus labels (assuming 10 genus labels)

params = (0.1, 0.5, 100, 1.0, 0.2, 0.01, 10, 50, True)
if params[-1]:
    fin = [list(range(x_tr.shape[1]))] * params[-3]
else:
    fin = [np.random.permutation(x_tr.shape[1]).tolist()[:params[-3]] for _ in range(params[-3])]
# Call the Bayesian_cls function with your fabricated data and hyperparameters
seen_acc, unseen_acc, H, acc_per_sclass, acc_per_usclass = Bayesian_cls(x_tr, y_tr, x_ts_us, y_ts_us, x_ts_s, y_ts_s, G, *params)

# Print the results
print("Seen Accuracy:", seen_acc)
print("Unseen Accuracy:", unseen_acc)
print("Harmonic Mean (H):", H)
print("Accuracy per Seen Class:", acc_per_sclass)
print("Accuracy per Unseen Class:", acc_per_usclass)


Model training time:
0.0008711814880371094
Model inference time:
0.00041103363037109375


IndexError: ignored